<a href="https://colab.research.google.com/github/SheidaTalei/FinalProject/blob/main/SheidaTalei_BirchVsTFIDF_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SUBJECT: Birch Vs TF-IDF
###AUTHOR: Sheida Talei

In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileExistsError: ignored

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.cluster import Birch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

In [2]:
#------------------------------------------Empty Rows Removal----------------------------------------------------------------
# This function Will: 1- Remove all empty rows from csv file 2- Save data to the same csv
def removeEmptyRows(fileName):
    df = pd.read_csv(fileName, encoding='utf-8-sig')
    df = df.dropna(subset=['text'], how='all', axis=0) 
    df.to_csv(fileName, header=True, encoding='utf-8-sig',  index=False)
    
    return df

In [3]:

X_and_Y = removeEmptyRows ('/content/drive/MyDrive/Final/train_temp.csv')
Y= X_and_Y.Label
X = X_and_Y.text

In [4]:
#--------------------------------------------------Loading StopWords ------------------------------------------
#Source of file: https://sites.google.com/site/kevinbouge/stopwords-lists
def getStopWord ():
    try:
        file = open('/content/drive/MyDrive/Final/stopwords_fa.txt', 'r', encoding='utf-8-sig')
        file_readed = file.read()
    
    finally:
        file.close()
        
    stopWord_Set = set(file_readed.split())
    return stopWord_Set


In [5]:
persian_stop_word = list(getStopWord())

In [6]:
vectorizer = TfidfVectorizer(stop_words= persian_stop_word ,  max_features = 2000 ) 
X_train_vector  = vectorizer.fit_transform(X)

In [7]:
#The branching_factor defines the number of sub-clusters and threshold sets the limit between the sample and sub-cluster.
#Balanced Iterative Reducing and Clustering using Hierarchies, or BIRCH for short, deals with large datasets by first generating a more compact summary that retains as 
#much distribution information as possible, and then clustering the data summary instead of the original dataset.
#Source: https://towardsdatascience.com/machine-learning-birch-clustering-algorithm-clearly-explained-fb9838cbeed9
k= 5
brc = Birch(branching_factor=5, n_clusters=k, threshold=0.85, compute_labels=True)
brc.fit(X_train_vector)


Birch(branching_factor=5, compute_labels=True, copy=True, n_clusters=5,
      threshold=0.85)

In [8]:
clusters = brc.predict(X_train_vector)

In [9]:
print ("Clusters: ")
print(clusters)

Clusters: 
[0 1 0 ... 1 1 0]


In [10]:
labels = brc.labels_
print(labels)

[0 1 0 ... 1 1 0]


In [11]:
#source:https://ai.intelligentonlinetools.com/ml/tag/text-clustering/
silhouette_score = metrics.silhouette_score(X_train_vector, labels, metric='euclidean')

In [12]:
print ("Silhouette_score: ",silhouette_score )


Silhouette_score:  0.0012493005980760581


In [ ]:
SSE = []
for cluster in range(1,50):
    brc = Birch(branching_factor=5, n_clusters=cluster, threshold=0.85, compute_labels=True)
    brc.fit(X_train_vector)
    labels = brc.labels_
    SSE.append(metrics.silhouette_score(X_train_vector, labels, metric='euclidean'))
    print (cluster)

# converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,50), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette_score')